In [2]:
%load_ext autoreload
%autoreload 2
from phi.torch.flow import *
from tqdm.auto import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
def step(v, p, dt=1.):
    v = advect.semi_lagrangian(v, v, dt)
    v, _ = fluid.make_incompressible(v, (Obstacle(Box(x=(0, 40), y=(70, 110))), ), Solve('auto', 1e-5, 0, x0=p))
    return v

target_velocity = 1.0

def fluid_simulation(bottom_velocity):
    initial_pressure = None
    boundary = extrapolation.combine_sides(x=(0,0), y=(ZERO_GRADIENT, ZERO_GRADIENT))
    velocity = StaggeredGrid((0,0), boundary, x=50, y=50, bounds=Box(x=100, y=100))
    velocity.values._tensors = math.scatter(velocity.values._tensors[1], math.stack((0, math.range(1, 0, 50))))
    # velocity.values._tensors[1].y[0] = bottom_velocity
    # print(velocity, boundary)
    # print(velocity.at_centers().values.native(velocity.at_centers().values))

    for _ in tqdm(range(2)):
        velocity = step(velocity, initial_pressure)

    error = field.l2_loss(math.mean(velocity.vector['y'].y[-1].values) - target_velocity)

    return error

print(fluid_simulation(0.5))
sim_grad = field.functional_gradient(fluid_simulation, wrt='bottom_velocity', get_output=True)
tensor = math.tensor(0.5)
sim_grad(tensor)

NotImplementedError: Tensors are not editable to preserve the autodiff chain. This feature might be added in the future. To update part of a tensor, use math.where() or math.scatter()